In [ ]:
import pydicom
import numpy as np
import os
from collections import deque

def hu_degerine_donustur(dicom_goruntusu):
    goruntu = dicom_goruntusu.pixel_array
    egim = dicom_goruntusu.RescaleSlope
    kesme_noktasi = dicom_goruntusu.RescaleIntercept
    goruntu_hu = goruntu * egim + kesme_noktasi
    return goruntu_hu

def klasorden_dcm_okuma(klasor_yolu):
    dcm_dosyalari = [os.path.join(klasor_yolu, f) for f in os.listdir(klasor_yolu) if f.endswith('.dcm')]
    if not dcm_dosyalari:
        print("Belirtilen klasörde DICOM dosyası bulunamadı.")
        return None

    dicom_dosyalari = [pydicom.dcmread(f) for f in dcm_dosyalari]
    dicom_dosyalari.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    hu_goruntuleri = [hu_degerine_donustur(dcm) for dcm in dicom_dosyalari]
    volume_3d = np.stack(hu_goruntuleri, axis=0)
    return volume_3d

def find_region_voxels_and_center(ct_volume, start_coords, hu_range):
    z_max, y_max, x_max = ct_volume.shape
    start_x, start_y, start_z = start_coords

    if not (0 <= start_z < z_max and 0 <= start_y < y_max and 0 <= start_x < x_max):
        print("Başlangıç koordinatları hacim sınırları dışında.")
        return None, None

    min_hu, max_hu = hu_range
    if not (min_hu <= ct_volume[start_z, start_y, start_x] <= max_hu):
        print(f"Başlangıç noktası HU aralığında değil: {ct_volume[start_z, start_y, start_x]}")
        return None, None

    queue = deque([(start_z, start_y, start_x)])
    mask = np.zeros_like(ct_volume, dtype=bool)
    mask[start_z, start_y, start_x] = True

    neighbors = [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]

    while queue:
        z, y, x = queue.popleft()
        for dz, dy, dx in neighbors:
            nz, ny, nx = z + dz, y + dy, x + dx
            if 0 <= nz < z_max and 0 <= ny < y_max and 0 <= nx < x_max and not mask[nz, ny, nx]:
                if min_hu <= ct_volume[nz, ny, nx] <= max_hu:
                    mask[nz, ny, nx] = True
                    queue.append((nz, ny, nx))

    region_voxels = np.argwhere(mask)
    if region_voxels.size == 0:
        print("HU aralığında bölge bulunamadı.")
        return None, None

    center = np.mean(region_voxels, axis=0) #voxel ortalaması (z,y,x)
    center_coords = (int(round(center[2])), int(round(center[1])), int(round(center[0])))  #(x,y,z)

    return region_voxels, center_coords

if __name__ == "__main__":
    dicom_klasor_yolu = r'D:\staj\AI\CT_Dicom\Second'
    baslangic_koordinati = (346, 321, 128)  #(x, y, z) 
    hu_araligi = (-200, 200)

    ct_volume = klasorden_dcm_okuma(dicom_klasor_yolu)

    if ct_volume is not None:
        voxeller, merkez = find_region_voxels_and_center(ct_volume, baslangic_koordinati, hu_araligi)
        if voxeller is not None and merkez is not None:
            print(f"Seçilen HU aralığındaki bölgedeki voxel sayısı: {voxeller.shape[0]}")
            print(f"Merkez koordinatları: (x={merkez[0]}, y={merkez[1]}, z={merkez[2]})")
        else:
            print("Bölge bulunamadı veya merkez hesaplanamadı.")
    else:
        print("CT hacmi yüklenemedi.")


Seçilen HU aralığındaki bölgedeki voxel sayısı: 395
Merkez koordinatları: (x=347, y=323, z=128)
